In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop,Adam
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from tensorflow.python.keras.models import Sequential
import tensorflow as tf
from tensorflow.python.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout,Activation

In [2]:
datadir="/home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4/image_data/train"
categories=["Jeans","Trouser"]

IMG_SIZE=200

In [3]:
#creating our training data for model training and also normalize the data for further processing.
training_data=[]

def create_training_data():
    for category in categories:
        path=os.path.join(datadir,category)
        class_num=categories.index(category)
        for img in os.listdir(path):
            try:
                
                img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass
create_training_data()
 
    

In [4]:
data_dir="/home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4/image_data/validation"
categories=["Jeans","Trouser"]


In [5]:
#creating our training data for model training and also normalize the data for further processing.
validation_data=[]

def create_validation_data():
    for category in categories:
        path=os.path.join(data_dir,category)
        class_num=categories.index(category)
        for img in os.listdir(path):
            try:
                
                img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                validation_data.append([new_array,class_num])
            except Exception as e:
                pass
create_validation_data()
 
    

In [6]:
#from preventing the overfitting and underfitting we have to shuffle the data 
import random
random.shuffle(training_data)

In [7]:
x_train=[]
y_train=[]

In [8]:
#making x and y for training the model
for features,label in training_data:
    x_train.append(features)
    y_train.append(label)

In [9]:
x_validation=[]
y_validation=[]

In [10]:
#making x and y for training the model
for features,label in validation_data:
    x_validation.append(features)
    y_validation.append(label)

In [11]:
x_train=np.array(x_train)
x_validation=np.array(x_validation)

In [12]:
x_train=x_train.reshape(x_train.shape[0],IMG_SIZE,IMG_SIZE,1)
x_validation=x_validation.reshape(x_validation.shape[0],IMG_SIZE,IMG_SIZE,1)

In [13]:
x_train=x_train.astype('float32')
x_validation=x_validation.astype('float32')

In [14]:
x_train=x_train/255
x_validation=x_validation/255

In [15]:
num_class=2
y_train=tf.keras.utils.to_categorical(y_train,num_class)
y_validation=tf.keras.utils.to_categorical(y_validation,num_class)

In [16]:

batch_size=10
epochs=15
input_shape=(IMG_SIZE,IMG_SIZE,1)

In [17]:
#creating model
model=Sequential()
model.add(Conv2D(32,kernel_size=(5,5),strides=(1,1),
                activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(64,kernel_size=(5,5),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(num_class,activation='sigmoid'))

model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 196, 196, 32)      832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 94, 94, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 47, 47, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 141376)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                9048128   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1

In [18]:
from tensorflow.keras.optimizers import Adam,RMSprop,Nadam,SGD
#from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau


In [20]:
def adam():
    checkpoint=ModelCheckpoint('/home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4/mens_wear_adam_cnn.h5',monitor='val_loss',mode='min',save_best_only=True,
                           verbose=1)

    earlystop=EarlyStopping(monitor='val_loss',min_delta=0,patience=5,verbose=1,restore_best_weights=True)

    reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,verbose=1,min_delta=0.0001)

    callbacks=[earlystop,checkpoint,reduce_lr]
    
    model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
             metrics=['accuracy'])
    history=model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,callbacks=callbacks,
            validation_data=(x_validation,y_validation))
adam()

Epoch 1/15
20/20 [==============================] - ETA: 0s - loss: 0.2334 - accuracy: 0.9000
Epoch 00001: val_loss improved from inf to 0.11598, saving model to /home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4/mens_wear_adam_cnn.h5
20/20 [==============================] - 23s 1s/step - loss: 0.2334 - accuracy: 0.9000 - val_loss: 0.1160 - val_accuracy: 0.9444 - lr: 0.0010
Epoch 2/15
20/20 [==============================] - ETA: 0s - loss: 0.1464 - accuracy: 0.9450
Epoch 00002: val_loss did not improve from 0.11598
20/20 [==============================] - 24s 1s/step - loss: 0.1464 - accuracy: 0.9450 - val_loss: 0.2361 - val_accuracy: 0.8889 - lr: 0.0010
Epoch 3/15
20/20 [==============================] - ETA: 0s - loss: 0.1120 - accuracy: 0.9600
Epoch 00003: val_loss improved from 0.11598 to 0.01522, saving model to /home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4/mens_wear_adam_cnn.h5
20/20 [==============================] - 28s 1s/step - loss: 0.1120 - accuracy: 0.9600 - val_lo

In [26]:
def Rmsprop():
    checkpoint=ModelCheckpoint('/home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4/mens_wear_RMSprop_cnn.h5',monitor='val_loss',mode='min',save_best_only=True,
                           verbose=1)

    earlystop=EarlyStopping(monitor='val_loss',min_delta=0,patience=5,verbose=1,restore_best_weights=True)

    reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,verbose=1,min_delta=0.0001)

    callbacks=[earlystop,checkpoint,reduce_lr]
    model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.RMSprop(),
             metrics=['accuracy'])
    history=model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,callbacks=callbacks,
            validation_data=(x_validation,y_validation))
Rmsprop()    

Epoch 1/15
20/20 [==============================] - ETA: 0s - loss: 1.2444 - accuracy: 0.9050
Epoch 00001: val_loss improved from inf to 0.02503, saving model to /home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4/mens_wear_RMSprop_cnn.h5
20/20 [==============================] - 25s 1s/step - loss: 1.2444 - accuracy: 0.9050 - val_loss: 0.0250 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/15
20/20 [==============================] - ETA: 0s - loss: 0.0276 - accuracy: 0.9900
Epoch 00002: val_loss improved from 0.02503 to 0.00073, saving model to /home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4/mens_wear_RMSprop_cnn.h5
20/20 [==============================] - 27s 1s/step - loss: 0.0276 - accuracy: 0.9900 - val_loss: 7.2860e-04 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/15
20/20 [==============================] - ETA: 0s - loss: 0.0070 - accuracy: 0.9950
Epoch 00003: val_loss improved from 0.00073 to 0.00001, saving model to /home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4/mens_wear_RM

In [27]:
def Nadam():
    checkpoint=ModelCheckpoint('/home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4mens_wear_Nadam_cnn.h5',monitor='val_loss',mode='min',save_best_only=True,
                           verbose=1)

    earlystop=EarlyStopping(monitor='val_loss',min_delta=0,patience=5,verbose=1,restore_best_weights=True)

    reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,verbose=1,min_delta=0.0001)

    callbacks=[earlystop,checkpoint,reduce_lr]
    model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Nadam(),
             metrics=['accuracy'])
    history=model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,callbacks=callbacks,
            validation_data=(x_validation,y_validation))
Nadam()    

Epoch 1/15
20/20 [==============================] - ETA: 0s - loss: 0.0559 - accuracy: 0.9900
Epoch 00001: val_loss improved from inf to 0.00015, saving model to /home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4mens_wear_Nadam_cnn.h5
20/20 [==============================] - 40s 2s/step - loss: 0.0559 - accuracy: 0.9900 - val_loss: 1.4913e-04 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/15
20/20 [==============================] - ETA: 0s - loss: 0.0033 - accuracy: 1.0000
Epoch 00002: val_loss improved from 0.00015 to 0.00004, saving model to /home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4mens_wear_Nadam_cnn.h5
20/20 [==============================] - 49s 2s/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 3.9639e-05 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/15
20/20 [==============================] - ETA: 0s - loss: 4.2153e-04 - accuracy: 1.0000
Epoch 00003: val_loss improved from 0.00004 to 0.00001, saving model to /home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4mens_wear_N

In [28]:
def SGD_momentum():
    checkpoint=ModelCheckpoint('/home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4/mens_wear_SGD_momentum_cnn.h5',monitor='val_loss',mode='min',save_best_only=True,
                           verbose=1)

    earlystop=EarlyStopping(monitor='val_loss',min_delta=0,patience=5,verbose=1,restore_best_weights=True)

    reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,verbose=1,min_delta=0.0001)

    callbacks=[earlystop,checkpoint,reduce_lr]
    model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.SGD(lr=0.001,momentum=0.9),
             metrics=['accuracy'])
    history=model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,callbacks=callbacks,
            validation_data=(x_validation,y_validation))
SGD_momentum()    

Epoch 1/15
20/20 [==============================] - ETA: 0s - loss: 2.7376e-05 - accuracy: 1.0000
Epoch 00001: val_loss improved from inf to 0.00000, saving model to /home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4/mens_wear_SGD_momentum_cnn.h5
20/20 [==============================] - 40s 2s/step - loss: 2.7376e-05 - accuracy: 1.0000 - val_loss: 1.7042e-06 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/15
20/20 [==============================] - ETA: 0s - loss: 2.6048e-05 - accuracy: 1.0000
Epoch 00002: val_loss improved from 0.00000 to 0.00000, saving model to /home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4/mens_wear_SGD_momentum_cnn.h5
20/20 [==============================] - 40s 2s/step - loss: 2.6048e-05 - accuracy: 1.0000 - val_loss: 1.5675e-06 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/15
20/20 [==============================] - ETA: 0s - loss: 2.5278e-05 - accuracy: 1.0000
Epoch 00003: val_loss improved from 0.00000 to 0.00000, saving model to /home/sk-ji/Cont_ent/FlipNwork

In [29]:
def NAG():
    checkpoint=ModelCheckpoint('/home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4/mens_wear_NAG_cnn.h5',monitor='val_loss',mode='min',save_best_only=True,
                           verbose=1)

    earlystop=EarlyStopping(monitor='val_loss',min_delta=0,patience=5,verbose=1,restore_best_weights=True)

    reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,verbose=1,min_delta=0.0001)

    callbacks=[earlystop,checkpoint,reduce_lr]
    model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.SGD(lr=0.01,nesterov=0.9),
             metrics=['accuracy'])
    history=model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,callbacks=callbacks,
            validation_data=(x_validation,y_validation))
NAG()    

Epoch 1/15
20/20 [==============================] - ETA: 0s - loss: 2.2024e-05 - accuracy: 1.0000
Epoch 00001: val_loss improved from inf to 0.00000, saving model to /home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4/mens_wear_NAG_cnn.h5
20/20 [==============================] - 27s 1s/step - loss: 2.2024e-05 - accuracy: 1.0000 - val_loss: 1.0584e-06 - val_accuracy: 1.0000 - lr: 0.0100
Epoch 2/15
20/20 [==============================] - ETA: 0s - loss: 2.1579e-05 - accuracy: 1.0000
Epoch 00002: val_loss improved from 0.00000 to 0.00000, saving model to /home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4/mens_wear_NAG_cnn.h5
20/20 [==============================] - 33s 2s/step - loss: 2.1579e-05 - accuracy: 1.0000 - val_loss: 1.0231e-06 - val_accuracy: 1.0000 - lr: 0.0100
Epoch 3/15
20/20 [==============================] - ETA: 0s - loss: 2.1144e-05 - accuracy: 1.0000
Epoch 00003: val_loss improved from 0.00000 to 0.00000, saving model to /home/sk-ji/Cont_ent/FlipNwork/Assignments/Assig

In [30]:
#Loading the model

import tensorflow as tf 
classifier = tf.keras.models.load_model('/home/sk-ji/Cont_ent/FlipNwork/Assignments/Assign_4/mens_wear_NAG_cnn.h5')

In [31]:
import warnings
warnings.filterwarnings('ignore')
import cv2
import numpy as np

def draw_test(name,res,input_im):
    BLACK=[0,0,0]
    res=int(res)
    if res==0:
        pred="Jeans"
    if res==1:
        pred="Trouser"
    expanded_image=cv2.copyMakeBorder(input_im,0,0,0,imageL.shape[0]*3, cv2.BORDER_CONSTANT,value=BLACK)
    expanded_image=cv2.cvtColor(expanded_image,cv2.COLOR_GRAY2BGR)
    cv2.putText(expanded_image,str(pred),(100,100),cv2.FONT_HERSHEY_COMPLEX_SMALL,4,(1,60,0),5)
    cv2.imshow(name,expanded_image)
    
    
for i in range (0,10):
    rand=np.random.randint(0,len(x_train))
    input_im=x_train[rand]
    
    imageL=cv2.resize(input_im,None,fx=4,fy=4,interpolation=cv2.INTER_CUBIC)
    input_im=input_im.reshape(1,200,200,1)
    
    #get prediction
    res=str(classifier.predict_classes(input_im,1,verbose=0)[0])
    
    draw_test("Prediction",res,imageL)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [32]:
pwd

'/home/sk-ji/fliprobo-assign'